# Plot the results of WIT


## User Inputs

In [ ]:
# Enter the Name of the polygon you want to check:
# i.e., the 'UniqueID' column in the shapefile
polyName = 'Lake_Fitri'

In [ ]:
# Where are the .csv files stored?
filepath = "results/"

# do you want to export the plot?
export_plot = False

# If so, where?
output_dir = "results/"

### Run the cells below to produce a stack plot of the polygon

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [ ]:
#import the csv and prepare dataframe
df = pd.read_csv(filepath + polyName + ".csv")
df['Datetime'] = pd.to_datetime(df.Datetime)
df['Datetime'] = df.Datetime.dt.normalize()
df = df.set_index('Datetime')

In [ ]:
# generate plot
warnings.filterwarnings("ignore")
#set up color palette
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]       

#make a stacked area plot
plt.clf()
fig= plt.figure(figsize = (22,6))
plt.stackplot(df.index, 
              df.wofs_area_percent, 
              df.wet_percent, 
              df.green_veg_percent, 
              df.dry_veg_percent,
              df.bare_soil_percent,
              labels=['open water',
                      'wet',
                      'green veg',
                      'dry veg',
                      'bare soil',
                     ], colors=pal, alpha = 0.6)

#set axis limits to the min and max
plt.axis(xmin = df.index[0], xmax = df.index[-1], ymin = 0, ymax = 100)

#add a legend and a tight plot box
plt.legend(loc='lower left', framealpha=0.6)
plt.title('Fractional Cover, Wetness, and Water')
plt.tight_layout()
if export_plot:
    #save the figure
    plt.savefig(f'{output_dir}{polyName}.png')#, transparent=True)


## Animating WIT with true colour

In [ ]:
import datacube.utils.rio
import datacube
import fiona
from datacube.utils import geometry
from datacube.utils.dask import start_local_dask
from datacube.utils.rio import configure_s3_access
import dask
import os
import WITdrill
import sys
sys.path.append('../../../examples/Scripts')
import deafrica_plotting
import deafrica_datahandling
%load_ext autoreload
%autoreload 2

dc = datacube.Datacube(app='rgb_animation')

#This will speed up loading data
datacube.utils.rio.set_default_rio_config(aws='auto', cloud_defaults=True)

In [ ]:
# dask.config.set({"distributed.dashboard.link": 
#         os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')+"proxy/{port}/status"})

# client = start_local_dask(threads_per_worker=2, memory_limit='4G')
# display(client)

# creds = configure_s3_access(client=client, 
#                             region_name='auto')

In [ ]:
shp_path = 'shapes/test_africa_wetland.shp'
time = ('2013-04', '2019-06')
res = (-30, 30)

In [ ]:
with fiona.open(shp_path) as input:
    crs = geometry.CRS(input.crs_wkt)

x = fiona.open(shp_path)
feat = x[0]
# print(feat)
first_geom = feat['geometry']
geom = geometry.Geometry(first_geom, crs=crs)

In [ ]:
# deafrica_plotting.display_map(x=(lon-buff, lon+buff), y=(lat+buff, lat-buff))

In [ ]:
# Create a query object
queryC2 = {
    'geopolygon': geom,
    'time': (time),
    'resolution': res,
    'group_by': 'solar_day'
}

In [ ]:
crs = deafrica_datahandling.mostcommon_crs(dc=dc,
                                          product='usgs_ls8c_level2_2',
                                          query=queryC2)

col2 = deafrica_datahandling.load_masked_usgs(dc=dc, min_gooddata=0.9,
                                            products=['usgs_ls8c_level2_2'],
                                            output_crs=crs,
                                            measurements=['red', 'green', 'blue'],
                                            align=(15, 15),
                                            **queryC2)

In [ ]:
col2

In [ ]:
WITdrill.animated_timeseries_WIT(
    col2.rolling(time=3, center=True, min_periods=1).mean(),
    df.rolling(window=3, center=True, min_periods=1).mean(),
    output_dir+polyName+ "_WIT.gif",
    width_pixels=1000,
    interval=300,
    show_date=True,
    annotation_kwargs={'fontsize': 15},
    title= "Lake Fitri, Chad",
)

## generating lots of plots

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [ ]:
# Where are the .csv files stored?
filepath = "results/tanzania/"

# If so, where?
output_dir = "results/tanzania_plots/"

In [ ]:
csvs = os.listdir(filepath)
csvs.sort()


In [ ]:
def generate_plots(file):
    #import the csv and prepare dataframe
    df = pd.read_csv(filepath + file)
    polyName = file[:-4]
    df['Datetime'] = pd.to_datetime(df.Datetime)
    df['Datetime'] = df.Datetime.dt.normalize()
    df = df.set_index('Datetime')

    # generate plot
    warnings.filterwarnings("ignore")
    #set up color palette
    pal = [sns.xkcd_rgb["cobalt blue"],
           sns.xkcd_rgb["neon blue"],
           sns.xkcd_rgb["grass"],
           sns.xkcd_rgb["beige"],
           sns.xkcd_rgb["brown"]]       

    #make a stacked area plot
    plt.clf()
    fig= plt.figure(figsize = (22,6))
    plt.stackplot(df.index, 
                  df.wofs_area_percent, 
                  df.wet_percent, 
                  df.green_veg_percent, 
                  df.dry_veg_percent,
                  df.bare_soil_percent,
                  labels=['open water',
                          'wet',
                          'green veg',
                          'dry veg',
                          'bare soil',
                         ], colors=pal, alpha = 0.6)

    #set axis limits to the min and max
    plt.axis(xmin = df.index[0], xmax = df.index[-1], ymin = 0, ymax = 100)

    #add a legend and a tight plot box
    plt.legend(loc='lower left', framealpha=0.6)
    plt.title('Fractional Cover, Wetness, and Water')
    plt.tight_layout()

    #save the figure
    plt.savefig(output_dir + file[:-4] + '.png')#, transparent=True)

In [ ]:
for i in csvs[1:]:
    generate_plots(i)
